# Scraping IMDB Top Rated 1000 Movies From IMDB site

* Site: https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=1&ref_=adv_prv 
* Here we would scrap all the top 1000 top rated movies resides across multiple pages.

In [6]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd 
import matplotlib.pyplot as plt


In [36]:
movie_number_list = []
movie_names_list = []
movie_relase_year_list = []
movie_duration_list = []
movie_genre_list = []
movie_rating_list = []
movie_director_list = []
movie_star_list = []
movie_intro_list = []
IMDB_top_rated_movies = []

headers = {"Accept-Language": "en-US,en;q=0.5"}

try:
    for i in range(1,1001,100):
        site = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start={}&ref_=adv_nxt".format(i)
        IMDB_Movies_Page_Source = requests.get(site,headers = headers )
        IMDB_Movies_Page_Source.raise_for_status()
        soup = BeautifulSoup(IMDB_Movies_Page_Source.text,'html.parser')
        
        
        movie_blocks = soup.find_all(name = 'div',class_ = "lister-item mode-advanced")

        for movie_block in movie_blocks:
            movie_number = movie_block.find('div',class_ = "lister-item-content").h3.span.text
            movie_number_list.append(movie_number.replace('.',''))
            movie_name = movie_block.find('div',class_ = "lister-item-content").h3.a.text
            movie_names_list.append(movie_name)
            movie_relase_year = movie_block.find('div',class_ = "lister-item-content").h3.find('span',class_ = "lister-item-year text-muted unbold").text
            movie_relase_year_list.append(movie_relase_year.replace('(','').replace(')',''))

            movie_duration = movie_block.find('div',attrs= {'class': "lister-item-content"}).p.find('span',class_ = "runtime").text
            movie_duration_list.append(movie_duration)
            movie_genre = movie_block.find('div',attrs= {'class': "lister-item-content"}).p.find('span',class_ = "genre").text
            movie_genre_list.append(movie_genre.strip())

            movie_rating = movie_block.find('div',class_ = "lister-item-content").find('div',class_ = 'ratings-bar').find('div',class_ = "inline-block ratings-imdb-rating").text
            movie_rating_list.append(movie_rating.strip())

            movie_intro = movie_block.find('div',class_ = "lister-item-content").find_all('p',class_ = "text-muted")[1].text
            movie_intro_list.append(movie_intro.lstrip())

        #     movie_director = movie_block.find('div',class_ = "lister-item-content").find_all('p')[2].a.text
        #     movie_director_list.append(movie_director)

            movie_stars_block = movie_block.find('div',class_ = "lister-item-content").find_all('p')[2].find_all('a')

            counter = 0
            moive_star_cast_list = []
            for a in movie_stars_block:
                if counter <1:
                    director_name = a.text
                    movie_director_list.append(director_name)
                    #print("director name", director_name)
                else:
                    moive_star_cast_list.append(a.text)
                    movie_star_list.append(a.text)

                    #print('star cast', a.text)

                counter+=1

            movie_data = {"Movie Number" : movie_number.replace('.',''),
                    "Movie Name"   : movie_name,
                    "Movie Year" : movie_relase_year.replace('(','').replace(')',''),
                    "Movie Duration" : movie_duration,
                    "Movie Genre" : movie_genre.strip(),
                    "Movie Rating": movie_rating.strip(),
                    "Movie Director" : director_name,
                    "Movie Star Cast": moive_star_cast_list,
                    "Movie Description": movie_intro.lstrip()

                    }
            IMDB_top_rated_movies.append(movie_data)

#         print(movie_number_list)
#         print(movie_relase_year_list)
#         print(movie_relase_year_list)
#         print(movie_duration_list)
#         print(movie_genre_list)
#         print(movie_rating_list)
#         print(movie_intro_list)
#         print(movie_director_list)
#         print(movie_star_list)
#         print(IMDB_top_rating_movies)

except Exception as e:
    print(e)
    
 




## Importing IMDB top rated movies into Pandas Dataframe and extracting it to csv file.

In [37]:
df = pd.DataFrame(IMDB_top_rated_movies)

file_save_path = r"C:\Users\sangarg\Desktop\Python\Small Project\IMDB_top_rated_movies.csv"
df.to_csv(file_save_path, index = False, encoding= 'utf-8')


In [38]:
df

,Movie Number,Movie Name,Movie Year,Movie Duration,Movie Genre,Movie Rating,Movie Director,Movie Star Cast,Movie Description
0,1,The Shawshank Redemption,1994,142 min,Drama,9.3,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Two imprisoned men bond over a number of years...
1,2,The Godfather,1972,175 min,"Crime, Drama",9.2,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",The aging patriarch of an organized crime dyna...
2,3,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",When the menace known as the Joker wreaks havo...
3,4,The Lord of the Rings: The Return of the King,2003,201 min,"Action, Adventure, Drama",9.0,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O...",Gandalf and Aragorn lead the World of Men agai...
4,5,Schindler's List,1993,195 min,"Biography, Drama, History",9.0,Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car...","In German-occupied Poland during World War II,..."
...,...,...,...,...,...,...,...,...,...
995,996,Sabrina,1954,113 min,"Comedy, Drama, Romance",7.6,Billy Wilder,"[Humphrey Bogart, Audrey Hepburn, William Hold...",A playboy becomes interested in the daughter o...
996,997,From Here to Eternity,1953,118 min,"Drama, Romance, War",7.6,Fred Zinnemann,"[Burt Lancaster, Montgomery Clift, Deborah Ker...","At a U.S. Army base in 1941 Hawaii, a private ..."
997,998,Snow White and the Seven Dwarfs,1937,83 min,"Animation, Adventure, Family",7.6,William Cottrell,"[David Hand, Wilfred Jackson, Larry Morey, Per...",Exiled into the dangerous forest by her wicked...
998,999,The 39 Steps,1935,86 min,"Crime, Mystery, Thriller",7.6,Alfred Hitchcock,"[Robert Donat, Madeleine Carroll, Lucie Mannhe...",A man in London tries to help a counter-espion...


# Data Cleaning

In [58]:
# Setting up a index for the dataset

df.set_index('Movie Number',inplace= True)

In [80]:
## cleaning Movie Year column for the unwanted charaters and changing the data type to intergers from object
df['Movie Year'] = df['Movie Year'].str.replace('I','').replace('II','').replace('III','').astype(int)

In [86]:
df.drop(columns = ['Movie Year 1','Movie Release Year'],inplace= True)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1,000
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Movie Name         1000 non-null   object
 1   Movie Year         1000 non-null   int32 
 2   Movie Duration     1000 non-null   object
 3   Movie Genre        1000 non-null   object
 4   Movie Rating       1000 non-null   object
 5   Movie Director     1000 non-null   object
 6   Movie Star Cast    1000 non-null   object
 7   Movie Description  1000 non-null   object
dtypes: int32(1), object(7)
memory usage: 98.7+ KB


In [89]:
df['Movie Rating'] = df['Movie Rating'].astype('float')

In [91]:
df['Movie Rating'].dtype

dtype('float64')

In [97]:
## setting up the rows display options to all the rows so that I can check the data easily
pd.set_option('display.max_rows', 1000)

In [99]:
df["Movie Duration"] = df["Movie Duration"].str.replace('min','').astype('int')
df

,Movie Name,Movie Year,Movie Duration,Movie Genre,Movie Rating,Movie Director,Movie Star Cast,Movie Description
Movie Number,,,,,,,,
1,The Shawshank Redemption,1994,142,Drama,9.3,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Two imprisoned men bond over a number of years...
2,The Godfather,1972,175,"Crime, Drama",9.2,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K...",The aging patriarch of an organized crime dyna...
3,The Dark Knight,2008,152,"Action, Crime, Drama",9.0,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",When the menace known as the Joker wreaks havo...
4,The Lord of the Rings: The Return of the King,2003,201,"Action, Adventure, Drama",9.0,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O...",Gandalf and Aragorn lead the World of Men agai...
5,Schindler's List,1993,195,"Biography, Drama, History",9.0,Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car...","In German-occupied Poland during World War II,..."
6,The Godfather: Part II,1974,202,"Crime, Drama",9.0,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia...",The early life and career of Vito Corleone in ...
7,12 Angry Men,1957,96,"Crime, Drama",9.0,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John...",The jury in a New York City murder trial is fr...
8,Jai Bhim,2021,164,"Crime, Drama, Mystery",8.9,T.J. Gnanavel,"[Suriya, Lijo Mol Jose, Manikandan K., Rajisha...",When a tribal man is arrested for a case of al...
9,Pulp Fiction,1994,154,"Crime, Drama",8.9,Quentin Tarantino,"[John Travolta, Uma Thurman, Samuel L. Jackson...","The lives of two mob hitmen, a boxer, a gangst..."


In [100]:
## column rename

df.rename(columns= {"Movie Duration": "Movie Duration in Minutes"}, inplace= True) 

In [101]:
df.describe()

,Movie Year,Movie Duration in Minutes,Movie Rating
count,1000.000000,1000.000000,1000.000000
mean,1991.190000,123.728000,7.962900
std,23.924972,28.477658,0.276822
min,1920.000000,45.000000,7.600000
25%,1975.000000,103.000000,7.700000
50%,1999.000000,120.000000,7.900000
75%,2010.000000,138.000000,8.100000
max,2022.000000,321.000000,9.300000
